<a href="https://colab.research.google.com/github/mehdihosseinimoghadam/RAG-AVA-Mistral-7B/blob/main/RAG_%2B_Mistral_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence-transformers metapub
!pip install accelerate
!pip install transformers accelerate bitsandbytes -q
! pip install transformers trl py7zr optimum accelerate bitsandbytes
! pip install git+https://github.com/huggingface/peft
! pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118

  Cloning https://github.com/huggingface/peft to /tmp/pip-req-build-ab3x9nbi
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft /tmp/pip-req-build-ab3x9nbi
  Resolved https://github.com/huggingface/peft to commit 01275b4cb3d70daa7ce52e261cf1c21e3adc38eb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118
  Using cached https://huggingface.github.io/autogptq-index/whl/cu118/auto-gptq/auto_gptq-0.7.1%2Bcu118-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (23.7 MB)
  Using cached rouge-1.0.1-py3-none-any.whl.metadata (4.1 kB)
  Using cached gekko-1.2.1-py3-none-any.whl.metadata (3.0 kB)
Using cached gekko-1.2.1-py3-none-any.whl (13.2 MB)
Using cached rouge-1.0.1-py3-none-any.whl (13 kB)


In [2]:
pip install faiss-cpu


In [3]:
pip install bitsandbytes

In [4]:
import faiss
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import numpy as np
import time

# Load your corpus from a CSV file
paper_text_df = pd.read_csv('/content/orders.csv').dropna()
print("Columns in the DataFrame:", paper_text_df.columns)
print("First few rows of the DataFrame:", paper_text_df.head())

# Select and preprocess text data
paper_text = (paper_text_df['orderid'].astype(str) + ' ' +
              paper_text_df['customerid'] + ' ' +
              paper_text_df['shipname'] + ' ' +
              paper_text_df['shipcity'] + ' ' +
              paper_text_df['shipcountry']).tolist()

# Check GPU availability
if not torch.cuda.is_available():
    print("Warning: No GPU detected. Processing will be slow. Please add a GPU to this notebook")

# Initialize SentenceTransformer model
model_name = 'LABSE'
model1 = SentenceTransformer(model_name)

# Encode the corpus
print("Encoding the corpus. This might take a while")
corpus_embeddings = model1.encode(paper_text, show_progress_bar=True, convert_to_tensor=True)

# Convert embeddings to numpy array for FAISS
corpus_embeddings_np = corpus_embeddings.cpu().numpy()

# Create and populate FAISS index
dimension = corpus_embeddings_np.shape[1]  # Dimensionality of the embeddings
index = faiss.IndexFlatL2(dimension)  # Using L2 distance (Euclidean distance)
index.add(corpus_embeddings_np)  # Add embeddings to the index


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Columns in the DataFrame: Index(['orderid', 'customerid', 'employeeid', 'orderdate', 'requireddate',
       'shippeddate', 'shipvia', 'freight', 'shipname', 'shipcity',
       'shippostalcode', 'shipcountry'],
      dtype='object')
First few rows of the DataFrame:    orderid customerid  employeeid   orderdate requireddate shippeddate  \
0    10248      VINET           5  04/07/1996   01/08/1996  16/07/1996   
1    10249      TOMSP           6  05/07/1996   16/08/1996  10/07/1996   
2    10250      HANAR           4  08/07/1996   05/08/1996  12/07/1996   
3    10251      VICTE           3  08/07/1996   05/08/1996  15/07/1996   
4    10252      SUPRD           4  09/07/1996   06/08/1996  11/07/1996   

   shipvia  freight                   shipname        shipcity shippostalcode  \
0        3    32.38  Vins et alcools Chevalier           Reims          51100   
1        1    11.61         Toms Spezialitäten         Münster          44087   
2        2    65.83              Hanari Carnes 

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/sentence_transformers/models/Dense.py:89: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(input_path, "pytorch_

Encoding the corpus. This might take a while


Batches:   0%|          | 0/25 [00:00<?, ?it/s]

In [9]:
def search_faiss(inp_question, num_res):
    res = {}
    start_time = time.time()
    question_embedding = model1.encode(inp_question, convert_to_tensor=True).cpu().numpy()
    # Reshape the question embedding to a 2D array
    question_embedding = question_embedding.reshape(1, -1)  # Reshape to (1, dimension)
    distances, indices = index.search(question_embedding, num_res)  # Search using FAISS
    end_time = time.time()

    print("Input question:", inp_question)
    print("Results (after {:.3f} seconds):".format(end_time - start_time))

    for idx in indices[0]:
        res[idx] = paper_text[idx]

    df = pd.DataFrame(list(res.items()), columns=['id', 'res'])
    return df

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Specify the model name or ID
model_name_or_id = "NousResearch/Nous-Hermes-2-Mistral-7B-DPO"

# Set device to CUDA if available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define quantization configuration without `load_in_8bit_fp32_cpu_offload`
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True  # Use 8-bit quantization
)

# Define custom device map for better control
device_map = {
    "": device
}

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_id,
    quantization_config=quantization_config,  # Pass the quantization configuration
    torch_dtype=torch.float16,  # Use float16 for efficiency
    device_map=device_map,      # Map model to specified device
    low_cpu_mem_usage=True      # Reduce CPU memory usage
)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_id)

print("Model and tokenizer loaded successfully.")


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model and tokenizer loaded successfully.


In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer
q = 'How many orders were shipped by each shipping company?'

res = search_faiss(q, 4)


prompt = f'''

answer the question based on the provided context:

{q},

context: {res['res'][0]} - {res['res'][1]} - {res['res'][2]} - {res['res'][3]}

'''


prompt = f"### Human:{prompt}\n### Assistant:"


inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.99,
    max_new_tokens=900,
    pad_token_id=tokenizer.eos_token_id
)


outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Input question: How many orders were shipped by each shipping company?
Results (after 0.031 seconds):
### Human:

answer the question based on the provided context:

How many orders were shipped by each shipping company?,

context: 10342 FRANK Frankenversand München Germany - 10670 FRANK Frankenversand München Germany - 10675 FRANK Frankenversand München Germany - 10472 SEVES Seven Seas Imports London UK


### Assistant:

The number of orders shipped by each shipping company is as follows:

1. Frankenversand München Germany: 3 orders (10670, 10675, and 10342)
2. Seven Seas Imports London UK: 1 order (10472)
